## Importing the Libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

## Importing the dataset

In [2]:
dataset = pd.read_csv('C:/Users/H_jam/OneDrive/Uni-MA/Term2/DataMining/DataSets/DeepLearning/Artificial_Neural_Networks/Churn_Modelling.csv')


X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values

## Encoding Categorical Data

In [3]:
from sklearn.preprocessing import LabelEncoder
labelencoder_X_2 = LabelEncoder()
X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2])

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
onehotencoder = ColumnTransformer([('one_hot_encoder',OneHotEncoder(categories= 'auto'),[1])], remainder = 'passthrough')
X = onehotencoder.fit_transform(X)
X = X[:, 1:]
X = np.array(X, dtype = float)

In [4]:
# Encoding Categorical Data

from sklearn.preprocessing import LabelEncoder
labelencoder_X_2 = LabelEncoder()
X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2])

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
onehotencoder = ColumnTransformer([('one_hot_encoder',OneHotEncoder(categories= 'auto'),[1])], remainder = 'passthrough')
X = onehotencoder.fit_transform(X)
X = X[:, 1:]
X = np.array(X, dtype = float)

In [5]:
# Splitting the dataset into the Training set and Test set

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

#unique, counts = np.unique(y_test, return_counts=True)
#dict(zip(unique, counts))

In [6]:
# Feature Scaling

from sklearn.preprocessing import StandardScaler

sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)


# Part 2 - Making ANN

In [7]:
# Importing the keras and packages

import keras
from keras.models import Sequential
from keras.layers import Dense

## ایجاد شبکه

In [8]:
classifier = Sequential()

In [9]:
# Adding the input layer and first hidden layer

classifier.add(Dense(units = 6, activation = 'relu', kernel_initializer= 'uniform', input_dim = 11))
# ورودی ها همون تعداد فیچرهاست

In [10]:
# Adding the second hidden layer

classifier.add(Dense(units = 6, activation = 'relu', kernel_initializer= 'uniform'))

In [11]:
# Adding the output layer

classifier.add(Dense(units = 1, activation = 'sigmoid', kernel_initializer= 'uniform'))

## Optimizer and Loss Function

In [12]:
# compile the ANN

classifier.compile(optimizer= 'adam', loss = 'binary_crossentropy', metrics= ['accuracy'])

In [13]:
# Fitting the ANN to the Training set

classifier.fit(X_train, y_train, batch_size= 10, epochs = 20)

Epoch 1/20
800/800 [==============================] - 1s 688us/step - loss: 0.4869 - accuracy: 0.7960
Epoch 2/20
800/800 [==============================] - 1s 686us/step - loss: 0.4275 - accuracy: 0.7960
Epoch 3/20
800/800 [==============================] - 1s 671us/step - loss: 0.4213 - accuracy: 0.8020
Epoch 4/20
800/800 [==============================] - 1s 664us/step - loss: 0.4173 - accuracy: 0.8207
Epoch 5/20
800/800 [==============================] - 1s 673us/step - loss: 0.4151 - accuracy: 0.8265
Epoch 6/20
800/800 [==============================] - 1s 665us/step - loss: 0.4133 - accuracy: 0.8303
Epoch 7/20
800/800 [==============================] - 1s 673us/step - loss: 0.4116 - accuracy: 0.8331
Epoch 8/20
800/800 [==============================] - 1s 659us/step - loss: 0.4101 - accuracy: 0.8328
Epoch 9/20
800/800 [==============================] - 1s 666us/step - loss: 0.4089 - accuracy: 0.8336
Epoch 10/20
800/800 [==============================] - 1s 684us/step - loss: 0.407

In [14]:
# Part 3 -Making the predictions and evaluating the model

# Predicting the Test set results

y_pred = classifier.predict(X_test)
y_pred = (y_pred >0.5)

63/63 [==============================] - 0s 606us/step


## Making the Confusion Matrix

In [15]:

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

## Grid Search

In [16]:
from keras.wrappers.scikit_learn import KerasClassifier
def build_clf(unit):
  # creating the layers of the NN
  ann = keras.models.Sequential()
  ann.add(keras.layers.Dense(units=unit, activation='relu'))
  ann.add(keras.layers.Dense(units=unit, activation='relu'))
  ann.add(keras.layers.Dense(units=1, activation='sigmoid'))
  ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
  return ann

model = KerasClassifier(build_fn=build_clf)

from sklearn.model_selection import GridSearchCV
params={'batch_size':[100, 20, 50], 
        'nb_epoch':[20, 10],
        'unit':[5, 10, 15],
           
        }
gs=GridSearchCV(estimator=model, param_grid=params, cv=10, scoring= 'accuracy')

C:\Users\H_jam\AppData\Local\Temp\ipykernel_23140\3298723986.py:11: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn=build_clf)


now fit the dataset to the GridSearchCV object. 


In [17]:
# now fit the dataset to the GridSearchCV object. 
gs = gs.fit(X_train, y_train)

best_params=gs.best_params_
accuracy=gs.best_score_
print(best_params)
print(accuracy)

25/25 [==============================] - 0s 564us/step


KeyboardInterrupt: 

In [ ]:
# best epoch
from keras.callbacks import ModelCheckpoint
mc = ModelCheckpoint('best', monitor = 'val_accuracy', mode = 'max', save_best_only = True, verbose = 0)

classifier.compile(optimizer= 'adam', loss = 'binary_crossentropy', metrics= ['accuracy'])

print('Training model ...')

r = classifier.fit(X_train, y_train, epochs = 20, batch_size = 64, validation_data = (X_test, y_test), callbacks = [mc])
from keras.models import load_model
best = load_model('best')


# plot loss per iteration
plt.plot(r.history['loss'], label = 'loss')
plt.plot(r.history['val_loss'], label = 'val_loss')
plt.legend()

# plot accuracy per iteration

plt.plot(r.history['accuracy'], label = 'acc')
plt.plot(r.history['val_accuracy'], label = 'val_acc')
plt.legend()

# Check the best obtained model

best.evaluate(X_test, y_test)